In [1]:
import pandas as pd
import time
from Functions import xbrl_dict,filings_collector
import datetime as dt
import requests
import xml.etree.ElementTree as ET
from pymongo import MongoClient

In [2]:
ticker = input('ticker?')

ticker?ba


In [3]:
lxml_link = 'https://www.sec.gov/Archives/edgar/data/12927/000001292713000041/ba-20130930.xml'

In [4]:
lxml_dict = xbrl_dict(lxml_link)

In [5]:
lxml_dict

{'company_name': 'BOEING CO',
 'CIK': '0000012927',
 'doc_date': datetime.datetime(2013, 9, 30, 0, 0),
 'doc_year': '2013',
 'doc_period': 'Q3',
 'common_shares_outstanding': 751453447,
 'amendment_flag': 'false',
 'exceptions': 1}

In [ ]:
xbrl_resp = requests.get(lxml_link)
xbrl_str = xbrl_resp.text
root = ET.fromstring(xbrl_str)
uri_list = []
names = []
sec_list = []
for child in root:
    uri_pos = child.tag.index('}')
    uri = ''.join(child.tag[1:uri_pos])
    name = ''.join(child.tag[int(uri_pos+1):len(child.tag)])

    if uri not in uri_list:
        uri_list.append(uri)

    if uri[0:22] == 'http://xbrl.us/us-gaap' or uri[0:23] == 'http://fasb.org/us-gaap':
        ref = child.attrib['contextRef']
        keys_test = {'ref' : ref, 'name' : name, 'data' : child.text}
        names.append(keys_test)  
    if uri[0:18] == 'http://xbrl.us/dei' or uri[0:23] == 'http://xbrl.sec.gov/dei':
        sec_keys = {'name' : name, 'attribs' : child.attrib, 'data' : child.text}
        sec_list.append(sec_keys)

In [ ]:
uri_list

In [ ]:
# names
q1_names_df = pd.DataFrame(names)
q1_names_df.head()

In [ ]:
q1_names_df.to_csv(f'../data/{ticker}_xml_test.csv', index=False)